In [1]:
import os
import re
import json
import urllib
from pathlib import Path
from bs4 import BeautifulSoup as soup
import http.cookiejar
import docx
import nltk
nltk.download('punkt')
import tldextract
import traceback
import warnings
warnings.filterwarnings('ignore')
import requests
from bs4 import BeautifulSoup

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import utils as util
import config_manager

In [3]:
serverConfig = config_manager.get_server_config()
storage = serverConfig['storage']
bucket_name = serverConfig['default_bucket']

In [4]:
def get_page_soup(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        # Send a GET request to the URL with headers
        response = requests.get(url, headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content using BeautifulSoup
            page_soup = BeautifulSoup(response.content, 'html.parser')
            return page_soup
        else:
            print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None


In [5]:
url = "https://www.esgtoday.com/climeworks-announces-direct-air-capture-technology-breakthrough-to-scale-carbon-removal/"
page_soup = get_page_soup(url)
website = "esgtoday"

In [6]:
def extractTitleAuthorDateContent(page_soup,articles_config,website):
    '''
    Using html tags from an article webpage, extracting title, author,published date and context from the articles
    '''
    response_obj = {"status": "success", "message": "", "file_path": "","file_size_in_kb":""}
    unwanted_class_names = ["author-bio__author-card__3h9uA","site-footer__content-container__3ASGG","media-story-card__body__3tRWy","mb-20 stryalsoread","text__text__1FZLe text__dark-grey__3Ml43 text__regular__2N1Xr text__ultra_small__37j9j body__base__22dCE body__ultra_small_body__1lUQl primary-gallery__caption__1UdH8","caption","attribution"]
    unwanted_tags = page_soup.find_all(class_=unwanted_class_names)
    for tag in unwanted_tags:
        tag.decompose()
    
    title = page_soup.findAll(articles_config[website]['title']['tag_name'], class_=articles_config[website]['title']['class_name']) if website == 'esgclarity' else page_soup.findAll(articles_config[website]['title']['tag_name'], class_=articles_config[website]['title']['class_name'])
    if website=="business-standard":
        extra_content= page_soup.findAll(articles_config[website]['extra']['tag_name'], class_=articles_config[website]['extra']['class_name'])
#         print(extra_content)
        extra_info = " ".join([element.get_text() for element in extra_content])
    else:
        extra_info=title[0].text.strip().split('\n')[-1].strip()
    title = title[0].text.split('\n')[1] if title != [] and website == "esgclarity" else title[0].text.strip() if title else ""
    print("Title : ", title)
    
    # Extracting Publisher name of the Article from the website
    if website == "theconversation":
        author_element = [a.findAll(articles_config[website]["author"]["tag_name_1"], class_=articles_config[website]["author"]["class_name_1"]) for a in page_soup.findAll(articles_config[website]["author"]["tag_name_2"],class_=articles_config[website]["author"]["class_name_2"]) if a.findAll(articles_config[website]["author"]["tag_name_3"])[0].text in articles_config[website]["author"]["text_list"]]
        author = ", ".join(name.text.strip() for name in author_element[0]) if author_element else ""
    elif website == "business-standard":
        author_element = [name.split("|")[0].strip() for sublist in page_soup.findAll(articles_config[website]['author']['tag_name'], class_= articles_config[website]['author']['class_name']) for name in sublist]
        author = ", ".join(name for name in author_element) if author_element else ""
    else:
        author_element = [name.text.strip() for name in page_soup.findAll(articles_config[website]['author']['tag_name'], class_=articles_config[website]['author']['class_name'])]
        author = ", ".join(name for name in author_element) if author_element else ""
    print("Author : ", author)
    # Extracting Published_date and contents of the Article from the websit
    if website == "reuters":
        date = page_soup.findAll(articles_config[website]['date']['tag_name'], class_=articles_config[website]['date']['class_name'])
        if date != []:
            date = date[0].text
        else:
            date = ""
#         paragraphs = [p.get_text(strip=True) for p in page_soup.findAll("p")]
        paragraphs = []
        for p in page_soup.findAll("p"):  
            if 'label' in str(p) or 'class' in p.attrs or 'style' in p.attrs or any(span_tag in p.descendants for span_tag in p.find_all('span')):
                text = ''
                for element in p.contents:
#                     print(element)
                    if isinstance(element, str):
                        text += element
                    elif element.name == 'a':
                        text += element.text
                if text:
                    paragraphs.append(text.strip())
        last_two_sentences = paragraphs[-2:]
        # print('last_two_sentences',last_two_sentences)
        if any("reporting by" in sentence.lower() for sentence in last_two_sentences):
            paragraphs = paragraphs[:-2]  # Remove last two sentences
        else:
            paragraphs = paragraphs[:-1]
    else:
        date_element = page_soup.find(articles_config[website]['date']['tag_name'], class_=articles_config[website]['date']['class_name'])
        if date_element != None:
            date_text_old = date_element.text
            regEx = r'(?:\d{1,2}[-/th|st|nd|rd\s]*)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\s,.]*(?:\d{1,2}[-/th|st|nd|rd)\s,]*)?(?:\d{2,4})'
            date_text_l = re.findall(regEx,date_text_old )
            date_text = date_text_l[0]
            date = date_text
        else:
            date = ""

        class_names = articles_config[website]['context'].get('class_names')
        if class_names:
            paragraphs = []
            strings_to_remove = ["Related Article:", "See related article:","To learn more, visit"]
            for class_name in class_names:
                paragraph_elements = page_soup.findAll(articles_config[website]['context']['tag_name'], class_=class_name)
                for p in paragraph_elements:
                    a_tag_target = p.find('a', href=True, target="_blank", rel="noreferrer noopener")
                    if a_tag_target:
                        a_tag_target.extract()
                        for string_to_remove in strings_to_remove:
                            p.string = p.get_text().replace(string_to_remove, "").strip()
                    em_tags = p.find_all('em')
                    # print(em_tags,"em_tag")
                    a_tag = p.find('a', href=True)
                    for em_tag in em_tags:
                        if a_tag and em_tag:
                            em_tag.extract()
                            a_tag.extract()
                    
                    strong_tags = p.find_all('strong')
                    # print('strong_tags',strong_tags,'strong_tags')
                    for strong_tag in strong_tags:
                        a_tag_inside_strong = strong_tag.find('a', href=True)
                        if a_tag_inside_strong:
                            strong_tag.extract()
                    
                    paragraphs.append(p.get_text().strip())
                # paragraphs.extend([p.text.strip() for p in paragraph_elements])
        else:
            paragraphs = []
        paragraphs = ' '.join(paragraphs)
        if title!=extra_info:
            paragraphs=extra_info+'\n'+paragraphs
        paragraphs = re.split('\n+', paragraphs)
    if paragraphs and ("source:" in paragraphs[-1].lower() or "esg clarity’fund" in paragraphs[-1].lower()):
        paragraphs = paragraphs[:-1]

    print("Date : ", date)    
    if len(paragraphs) >= 1:
        if paragraphs[0] != "":       
            title = title + "." if title and not title.endswith(".") else title
            author = author + "." if author and not author.endswith(".") else author
            date = date + "." if date and not date.endswith(".") else date
            if title:
                response_obj['title']=title
            else:
                response_obj['title']=""
                response_obj["message"] += "Title is empty."
                print(articles_config[website]['context']['class_name'],'title tag is not found')
            if author:
                response_obj['author'] = author
            else:
                response_obj['author'] =""
                response_obj["message"] += "author is empty." 
                print(articles_config[website]['context']['class_name'],'title tag is not found')
            if date:
                response_obj['date']= date
            else:
                response_obj['date']= ""
                response_obj["message"] += "date is empty."
                print(articles_config[website]['context']['class_name'],'date tag is not found')
            return response_obj,paragraphs

        else:
            response_obj["status"] = "error"
            response_obj["message"] = "No data found."
            print(articles_config[website]['context']['class_names'],'context tag is not found')
#             print(response_obj)
            return response_obj,""


In [6]:

f = open('articles_config.json')
articles_config = json.load(f)
user_id=0
user_type='private'

In [7]:
def extractTitleAuthorDateContent(page_soup, articles_config, website):
    '''
    Using html tags from an article webpage, extracting title, author, published date and content from the articles.
    '''
    response_obj = {"status": "success", "message": "", "file_path": "", "file_size_in_kb": ""}
    unwanted_class_names = ["footer-headline","source_link picCaption","author-bio__author-card__3h9uA", "site-footer__content-container__3ASGG", "media-story-card__body__3tRWy", "mb-20 stryalsoread", "text__text__1FZLe text__dark-grey__3Ml43 text__regular__2N1Xr text__ultra_small__37j9j body__base__22dCE body__ultra_small_body__1lUQl primary-gallery__caption__1UdH8", "caption", "attribution", "data-io-article-url"]
    unwanted_tags = page_soup.find_all(class_=unwanted_class_names)
    for tag in unwanted_tags:
        tag.decompose()

    title = page_soup.findAll(articles_config[website]['title']['tag_name'], class_=articles_config[website]['title']['class_name']) if website == 'esgclarity' else page_soup.findAll(articles_config[website]['title']['tag_name'], class_=articles_config[website]['title']['class_name'])
    if website == "business-standard":
        extra_content = page_soup.findAll(articles_config[website]['extra']['tag_name'], class_=articles_config[website]['extra']['class_name'])
        extra_info = " ".join([element.get_text() for element in extra_content])
    else:
        extra_info = title[0].text.strip().split('\n')[-1].strip() if title else ""
    title = title[0].text.split('\n')[1] if title != [] and website == "esgclarity" else title[0].text.strip() if title else ""
    print("Title : ", title)

    # Extracting Publisher name of the Article from the website
    if website == "theconversation":
        author_element = [a.findAll(articles_config[website]["author"]["tag_name_1"], class_=articles_config[website]["author"]["class_name_1"]) for a in page_soup.findAll(articles_config[website]["author"]["tag_name_2"], class_=articles_config[website]["author"]["class_name_2"]) if a.findAll(articles_config[website]["author"]["tag_name_3"])[0].text in articles_config[website]["author"]["text_list"]]
        author = ", ".join(name.text.strip() for name in author_element[0]) if author_element else ""
    elif website == "business-standard":
        author_element = [name.split("|")[0].strip() for sublist in page_soup.findAll(articles_config[website]['author']['tag_name'], class_=articles_config[website]['author']['class_name']) for name in sublist]
        author = ", ".join(name for name in author_element) if author_element else ""
    else:
        author_element = [name.text.strip() for name in page_soup.findAll(articles_config[website]['author']['tag_name'], class_=articles_config[website]['author']['class_name'])]
        author = ", ".join(name for name in author_element) if author_element else ""
    print("Author : ", author)

    # Extracting Published_date and contents of the Article from the website
    if website == "reuters":
        date = page_soup.findAll(articles_config[website]['date']['tag_name'], class_=articles_config[website]['date']['class_name'])
        if date != []:
            date = date[0].text
        else:
            date = ""
        paragraphs = []
        for p in page_soup.findAll("p"):
            if 'label' in str(p) or 'class' in p.attrs or 'style' in p.attrs or any(span_tag in p.descendants for span_tag in p.find_all('span')):
                text = ''
                for element in p.contents:
                    if isinstance(element, str):
                        text += element
                    elif element.name == 'a':
                        text += element.text
                if text:
                    paragraphs.append(text.strip())
        last_two_sentences = paragraphs[-2:]
        if any("reporting by" in sentence.lower() for sentence in last_two_sentences):
            paragraphs = paragraphs[:-2]
        else:
            paragraphs = paragraphs[:-1]
    else:
        date_element = page_soup.find(articles_config[website]['date']['tag_name'], class_=articles_config[website]['date']['class_name'])
        if date_element:
            date_text_old = date_element.text
            regEx = r'(?:\d{1,2}[-/th|st|nd|rd\s]*)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\s,.]*(?:\d{1,2}[-/th|st|nd|rd)\s,]*)?(?:\d{2,4})'
            date_text_l = re.findall(regEx, date_text_old)
            date_text = date_text_l[0] if date_text_l else ""
            date = date_text
        else:
            date = ""

        class_names = articles_config[website]['context'].get('class_names')
        if class_names:
            paragraphs = []
            strings_to_remove = ["Related Article:", "See related article:", "To learn more, visit"]
            for class_name in class_names:
                paragraph_elements = page_soup.findAll(articles_config[website]['context']['tag_name'], class_=class_name)
                for p in paragraph_elements:
                    a_tag_target = p.find('a', href=True, target="_blank", rel="noreferrer noopener")
                    if a_tag_target:
                        a_tag_target.extract()
                        for string_to_remove in strings_to_remove:
                            p.string = p.get_text().replace(string_to_remove, "").strip()
                    em_tags = p.find_all('em')
                    a_tag = p.find('a', href=True)
                    for em_tag in em_tags:
                        if a_tag and em_tag:
                            em_tag.extract()
                            a_tag.extract()
                    strong_tags = p.find_all('strong')
                    for strong_tag in strong_tags:
                        a_tag_inside_strong = strong_tag.find('a', href=True)
                        if a_tag_inside_strong:
                            strong_tag.extract()
                    paragraphs.append(p.get_text().strip())
        else:
            paragraphs = []
        paragraphs = ' '.join(paragraphs)
        if title != extra_info:
            paragraphs = extra_info + '\n' + paragraphs
        paragraphs = re.split('\n+', paragraphs)

    if website == "indiatimes":
        paragraphs = []
        for p in page_soup.find_all(['p', 'h4','strong']):
        #for p in page_soup.find_all('p'):
            if "Also Read:" not in p.get_text():
                paragraph_text = p.get_text()
                #strong_texts = ' '.join(strong.get_text() for strong in p.find_all('strong'))
                #if strong_texts:
                  #  paragraph_text += ' ' + strong_texts
                paragraphs.append(paragraph_text)
        #paragraphs = [p.get_text() + ' ' + ' '.join(strong.get_text() for strong in p.find_all('strong')) for p in page_soup.find_all('p') if "Also Read:" not in p.get_text()]
        
          #paragraphs = [p.get_text() for p in page_soup.find_all('p') if "Also Read:" not in p.get_text()]

    if paragraphs and ("source:" in paragraphs[-1].lower() or "esg clarity’fund" in paragraphs[-1].lower()):
        paragraphs = paragraphs[:-1]

    print("Date : ", date)
    if len(paragraphs) >= 1:
        if paragraphs[0] != "":
            title = title + "." if title and not title.endswith(".") else title
            author = author + "." if author and not author.endswith(".") else author
            date = date + "." if date and not date.endswith(".") else date
            if title:
                response_obj['title'] = title
            else:
                response_obj['title'] = ""
                response_obj["message"] += "Title is empty."
            if author:
                response_obj['author'] = author
            else:
                response_obj['author'] = ""
                response_obj["message"] += "Author is empty."
            if date:
                response_obj['date'] = date
            else:
                response_obj['date'] = ""
                response_obj["message"] += "Date is empty."
            return response_obj, paragraphs
        else:
            response_obj["status"] = "error"
            response_obj["message"] = "No data found."
            return response_obj, ""
extractTitleAuthorDateContent(page_soup, articles_config, website)


Title :  Climeworks Announces Direct Air Capture Technology Breakthrough to Scale Carbon Removal
Author :  Susan Lahey
Date :  June 5, 2024


({'status': 'success',
  'message': '',
  'file_path': '',
  'file_size_in_kb': '',
  'title': 'Climeworks Announces Direct Air Capture Technology Breakthrough to Scale Carbon Removal.',
  'author': 'Susan Lahey.',
  'date': 'June 5, 2024.'},
 ['Zurich-based carbon removal company Climeworks revealed today its Generation 3 direct air capture (DAC) technology, providing “breakthrough” improvements in efficiency and performance, and forming a step in the company’s plan to scale up to megaton carbon removal capacity.',
  'DAC technology, listed by the IEA as a key carbon removal option in the transition to a net-zero energy system, extracts CO2 directly from the atmosphere for use as a raw material or permanent removal when combined with storage. According to the landmark 2022 Intergovernmental Panel on Climate Change (IPCC) climate change mitigation study, scenarios that limit warming to 1.5°C include carbon dioxide removal methods scaling to billions of tons of removal annually over the

In [8]:
#extractTitleAuthorDateContent(page_soup,articles_config,website)

In [9]:
def extractedArticlesToTextFile(url, page_soup, articles_config, response_obj, user_id, user_type, storage='local'):
    '''
    Extracting Article Contents and saving as a Text File
    '''
    # "domain" to extract webpage name - used as a "key" in articles_config dictionary
    domain = tldextract.extract(url).registered_domain
    website = domain.split(".")[0]
    response_obj, paragraphs = extractTitleAuthorDateContent(page_soup, articles_config, website)
    
    if response_obj["status"] == "success":
        title = response_obj["title"]
        date = response_obj['date']
        author = response_obj['author']
        if website == "business-standard":
            content = '\n\n'.join(paragraphs)
        else:    
            content = '\n\n'.join(paragraphs[articles_config[website]["context"]["start_index"]:articles_config[website]["context"]["end_index"]]) if "end_index" in articles_config[website]["context"] else '\n\n'.join(paragraphs[articles_config[website]["context"]["start_index"]:])
        
        words_count = len(nltk.word_tokenize(content))
        doc = docx.Document()
        doc.add_paragraph(title)
        doc.add_paragraph(f"{words_count} words.")
        doc.add_paragraph(date)
        doc.add_paragraph(author)
        doc.add_paragraph(content)
        
        file_name_split = url.split("/")
        if file_name_split[-1] != '':
            file_name = file_name_split[-1]
        else:
            file_name = file_name_split[-2]
        
        author = author.split(", ")
        author = author[0]
        file_name = file_name_split[2].replace('www.', '').replace('.', '-') + '-' + ('-'.join(file_name.split('-')[:5])).title() + ".docx"
        
        if user_type == 'PRIVATE':
            file_name = str(user_id) + '_' + file_name
        
        if storage == 'local':
            # Use the user's home directory to avoid permission issues
            home_directory = Path.home()
            path = home_directory / "input_data"
            if not path.exists():
                path.mkdir(parents=True, exist_ok=True)
            file_path = path / file_name
            doc.save(file_path)
            size = os.path.getsize(file_path)
            size_in_kb = (size // 1024)
        else:
            file_path = "input_data/" + file_name
            # size_in_kb = aws.writeIntoBucket(s3_client, bucket_name, file_path, doc, 'doc')
        
        response_obj["file_path"] = str(file_path)
        response_obj["file_size_in_kb"] = size_in_kb
    else:
        response_obj["status"] = "error"
        response_obj["message"] = "No data found in {}".format(url)
    
    return response_obj

In [10]:
response_obj=extractTitleAuthorDateContent(page_soup,articles_config,website)

Title :  Climeworks Announces Direct Air Capture Technology Breakthrough to Scale Carbon Removal
Author :  Susan Lahey
Date :  June 5, 2024


In [11]:
extractedArticlesToTextFile(url,page_soup,articles_config,response_obj,user_id,user_type)

Title :  Climeworks Announces Direct Air Capture Technology Breakthrough to Scale Carbon Removal
Author :  Susan Lahey
Date :  June 5, 2024


{'status': 'success',
 'message': '',
 'file_path': 'C:\\Users\\admin\\input_data\\esgtoday-com-Climeworks-Announces-Direct-Air-Capture.docx',
 'file_size_in_kb': 37,
 'title': 'Climeworks Announces Direct Air Capture Technology Breakthrough to Scale Carbon Removal.',
 'author': 'Susan Lahey.',
 'date': 'June 5, 2024.'}

In [12]:
def extractArticleContentsMain(url,articles_config,user_id,user_type):
    '''
    Main Function to Get Request from the Webpage using BeautifulSoup and Extracting Contents from Article 
    '''
    response_obj = {"status": "success", "message": "", "file_path": "","file_size_in_kb":""}
    try:
        # To get request from the wepage url
        cookie_jar = http.cookiejar.CookieJar()
        opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cookie_jar))
        urllib.request.install_opener(opener)
        headers = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
            'Referer': url
        }

        req = urllib.request.Request(url, headers=headers)
        webpage = urllib.request.urlopen(req).read()
        page_soup = soup(webpage, "html.parser")
        
        # Get response_obj dict after writing the contents extracted from the Article webpage
        response_obj = extractedArticlesToTextFile(url,page_soup,articles_config,response_obj,user_id,user_type)
        return response_obj
    except urllib.error.HTTPError as e:
        response_obj["status"] = "error"
        response_obj["message"] = "Unable to process {}, try after sometime.".format(url)
            
    except Exception as e:
        print("Exception occurred extract_article**********",str(e))
        exception_error = ''.join(traceback.TracebackException.from_exception(e).format())
        filename = "extract_article_error_"+".txt"

        if os.path.exists(filename):
            with open(filename, "a") as file:
                file.write(f"{exception_error}\n")
        else:
            with open(filename, "w") as file:
                file.write(f"{exception_error}\n")    
        return response_obj


In [13]:
extractArticleContentsMain(url,articles_config,user_id,user_type)

Title :  Climeworks Announces Direct Air Capture Technology Breakthrough to Scale Carbon Removal
Author :  Susan Lahey
Date :  June 5, 2024


{'status': 'success',
 'message': '',
 'file_path': 'C:\\Users\\admin\\input_data\\esgtoday-com-Climeworks-Announces-Direct-Air-Capture.docx',
 'file_size_in_kb': 37,
 'title': 'Climeworks Announces Direct Air Capture Technology Breakthrough to Scale Carbon Removal.',
 'author': 'Susan Lahey.',
 'date': 'June 5, 2024.'}

In [14]:
def extractMulitLinks(multi_links,user_id,user_type):
    with open("articles_config.json") as f:
        articles_config = json.load(f)
    response_obj = {"status": "success", "message": "",  "files": []}
    
    try:
        if len(multi_links) >= 1 and multi_links[0] == "":
            response_obj["status"] = "error"
            response_obj["message"] = "No url found"
            return response_obj

        elif len(multi_links) > 5:
            response_obj["status"] = "error"
            response_obj["message"] = "url limit exceeded"
            return response_obj
            
        for i in multi_links:
            domain = tldextract.extract(i).registered_domain
            website = domain.split(".")[0]
            print("website : ", website)
            if website not in articles_config:
                response_obj["status"] = "error"
                response_obj["message"] = "Link source is not recognized. Support sites are: esgtoday.com, esgnews.com, esgclarity.com, reuters.com,theconversation.com, business-standard.com"
                return response_obj
            
            inner_dict = {}
            status = extractArticleContentsMain(i,articles_config,user_id,user_type)
            response_obj['message']=status['message']
            if status["status"] == "error":
                return status
            else:
                inner_dict["file_path"] = status["file_path"]
                inner_dict["file_size_in_kb"] = status["file_size_in_kb"]
                response_obj["files"].append(inner_dict)
        return response_obj
    except Exception as e:
        print("Exception occurred extract_article_content**********",str(e))
        exception_error = ''.join(traceback.TracebackException.from_exception(e).format())
        print(exception_error)
        response_object["message"] = "unable to process : {}".format(i) 
        return response_object


In [15]:
multi_links=['https://esgnews.com/china-dominates-global-wind-and-solar-energy-construction/','https://theconversation.com/young-people-led-surge-for-smaller-parties-but-no-reform-youthquake-says-uk-election-survey-234394','https://www.business-standard.com/india-news/eighty-million-new-jobs-in-last-three-to-four-years-says-pm-modi-124071300691_1.html','https://www.indiatimes.com/lifestyle/whats-cooking/ambani-wedding-kim-kardashian-khloe-kardashian-look-638072.html','https://www.indiatimes.com/health/healthyliving/why-you-should-stop-walking-barefoot-at-home-636547.html']

In [16]:
extractMulitLinks(multi_links,user_id,user_type)

website :  esgnews
Title :  China Dominates Global Wind and Solar Energy Construction
Author :  ESG News
Date :  July 12, 2024
website :  theconversation
Title :  Young people led surge for smaller parties but no Reform ‘youthquake’, says UK election survey
Author :  Stuart Fox
Date :  July 12, 2024
website :  business-standard
Title :  Eighty million new jobs in last three to four years, says PM Modi
Author :  Press Trust of India
Date :  Jul 13 2024
website :  indiatimes
Title :  Ambani wedding: Check out Kim K and Khloé Kardashian's desi Barbie looks, Internet is not feeling it
Author :  IT Lifestyle Desk
Date :  Jul 13, 2024
website :  indiatimes
Title :  6 reasons why you should stop walking barefoot at home right now
Author :  IT Lifestyle Desk
Date :  Jul 13, 2024


{'status': 'success',
 'message': '',
 'files': [{'file_path': 'C:\\Users\\admin\\input_data\\esgnews-com-China-Dominates-Global-Wind-And.docx',
   'file_size_in_kb': 37},
  {'file_path': 'C:\\Users\\admin\\input_data\\theconversation-com-Young-People-Led-Surge-For.docx',
   'file_size_in_kb': 38},
  {'file_path': 'C:\\Users\\admin\\input_data\\business-standard-com-Eighty-Million-New-Jobs-In.docx',
   'file_size_in_kb': 35},
  {'file_path': 'C:\\Users\\admin\\input_data\\indiatimes-com-Ambani-Wedding-Kim-Kardashian-Khloe.docx',
   'file_size_in_kb': 37},
  {'file_path': 'C:\\Users\\admin\\input_data\\indiatimes-com-Why-You-Should-Stop-Walking.docx',
   'file_size_in_kb': 37}]}

In [17]:
def file_present_in_db(url):
    file_name_split = url.split("/")
    if file_name_split[-1]!='':
        file_name = file_name_split[-1]
    else:
        file_name = file_name_split[-2]
    file_name = file_name_split[2].replace('www.','').replace('.','-')+'-'+('-'.join(file_name.split('-')[:5])).title()  + ".docx"
    return file_name

In [18]:
file_present_in_db(url)

'esgtoday-com-Climeworks-Announces-Direct-Air-Capture.docx'

In [19]:
def checkDocProcessed_links(url,user_id,user_type):
    file_name = file_present_in_db(url)
    if user_type == 'PRIVATE':
        file_name = str(user_id)+'_'+file_name
    #query = "select * from playground_doc_cache_info where doc_name = '" + file_name + "'"
    #sql_df = util.loadDataFromSqlToDf(serverConfig["mysql_config"], query)
    #if file_name in sql_df["doc_name"].tolist():
       # return file_name
    #else:
        #return None


In [20]:
def process_links(multi_links,user_id,user_type):
    response_obj = {"status": "success", "message": "", "files": []}
    for url in multi_links:
        # file_name = file_present_in_db(url)
        result = checkDocProcessed_links(url,user_id,user_type)
        if result is None:
            resp_object = extractMulitLinks([url],user_id,user_type) 
            response_obj["files"].append(resp_object["files"][0])
        else:
            inner_dict = {}
            file_name = file_present_in_db(url)
            if storage == 'local':
                p = Path(str(os.path.abspath(os.curdir)))
                path = p.parent
                file_path = str(path)+"/input_data/"+file_name
                size = os.path.getsize(file_path)
                size_in_kb = size // 1024
            else:
                file_path = "input_data/"+file_name
                size_in_kb = aws.getS3BucketFileSize(aws.S3_Client,bucket_name, file_path)
            inner_dict["file_path"] = file_name
            inner_dict["file_size_in_kb"] = size_in_kb
            response_obj["files"].append(inner_dict)
    return response_obj

In [21]:
process_links(multi_links,user_id,user_type)

website :  esgnews
Title :  China Dominates Global Wind and Solar Energy Construction
Author :  ESG News
Date :  July 12, 2024
website :  theconversation
Title :  Young people led surge for smaller parties but no Reform ‘youthquake’, says UK election survey
Author :  Stuart Fox
Date :  July 12, 2024
website :  business-standard
Title :  Eighty million new jobs in last three to four years, says PM Modi
Author :  Press Trust of India
Date :  Jul 13 2024
website :  indiatimes
Title :  Ambani wedding: Check out Kim K and Khloé Kardashian's desi Barbie looks, Internet is not feeling it
Author :  IT Lifestyle Desk
Date :  Jul 13, 2024
website :  indiatimes
Title :  6 reasons why you should stop walking barefoot at home right now
Author :  IT Lifestyle Desk
Date :  Jul 13, 2024


{'status': 'success',
 'message': '',
 'files': [{'file_path': 'C:\\Users\\admin\\input_data\\esgnews-com-China-Dominates-Global-Wind-And.docx',
   'file_size_in_kb': 37},
  {'file_path': 'C:\\Users\\admin\\input_data\\theconversation-com-Young-People-Led-Surge-For.docx',
   'file_size_in_kb': 38},
  {'file_path': 'C:\\Users\\admin\\input_data\\business-standard-com-Eighty-Million-New-Jobs-In.docx',
   'file_size_in_kb': 35},
  {'file_path': 'C:\\Users\\admin\\input_data\\indiatimes-com-Ambani-Wedding-Kim-Kardashian-Khloe.docx',
   'file_size_in_kb': 37},
  {'file_path': 'C:\\Users\\admin\\input_data\\indiatimes-com-Why-You-Should-Stop-Walking.docx',
   'file_size_in_kb': 37}]}

In [22]:
articles_config

{'esgtoday': {'title': {'tag_name': 'h1',
   'class_name': 'post-title entry-title'},
  'author': {'tag_name': 'span', 'class_name': 'author vcard'},
  'date': {'tag_name': 'time', 'class_name': 'post-date entry-date updated'},
  'context': {'tag_name': 'div',
   'class_names': ['entry-content'],
   'start_index': 0}},
 'esgnews': {'title': {'tag_name': 'h1', 'class_name': 'c-h1 tw-mb-[0.5em]'},
  'author': {'tag_name': 'a', 'class_name': 'tw-text-black hover:tw-text-red'},
  'date': {'tag_name': 'div', 'class_name': 'tw-p-2'},
  'context': {'tag_name': 'div',
   'class_names': ['simple-text tt-content title-droid margin-big tw-text-base tw-leading-relaxed'],
   'start_index': 1}},
 'theconversation': {'title': {'tag_name': 'h1',
   'class_name': 'legacy entry-title instapaper_title'},
  'author': {'tag_name_1': 'span',
   'class_name_1': 'fn author-name',
   'tag_name_2': 'div',
   'class_name_2': 'content-authors-group',
   'tag_name_3': 'h3',
   'text_list': ['Author', 'Authors']},
